In [339]:
import cv2
import numpy as np

In [340]:
def get_line_pixels_vertical(image, x0, y0, x1, y1):
    # Implement the Bresenham's line algorithm
    points = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    if dx > dy:
        err = dx / 2.0
        while x != x1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= x-1 < len(image[0]) and 0 <= x+1 < len(image[0]):
                    points.append((x, y))
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy / 2.0
        while y != y1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= x-1 < len(image[0]) and 0 <= x+1 < len(image[0]):
                    points.append((x, y))
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy
    if 0 <= x < len(image[0]) and 0 <= y < len(image):
        if 0 <= x-1 < len(image[0]) and 0 <= x+1 < len(image[0]):
            points.append((x, y))

    # Get the pixel values
    pixels = [(image[y][x-1], image[y][x+1]) for x, y in points]

    return pixels

In [341]:
# Define a function to calculate color contrast between two pixels
def color_contrast(pixel1, pixel2):
    return pixel1 - pixel2

In [342]:
# Load the image
image = cv2.imread('cropped/AA AT-096-[0].jpg', 0)  # 0 loads the image in grayscale


In [343]:
# Define the line parameters (angle and starting point)
angle = 98  # Replace with your desired angle in degrees
x0, y0 = 239, 0  # Replace with your desired starting point

# Convert the angle to radians
angle = np.radians(angle)

# Calculate the end point of the line
x1 = int(x0 + 200 * np.cos(angle))
y1 = int(y0 + 200 * np.sin(angle))

In [344]:
image2 = image.copy()

cv2.line(image2, (x0, y0), (x1, y1), 255, 1)
cv2.line(image2, (x0+1, y0), (x1+1, y1), 0, 1)
cv2.line(image2, (x0-1, y0), (x1-1, y1), 0, 1)

cv2.imshow('image', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [345]:
all_values = []
for angle in range(80, 100):
    all_contrasts = []

    # print("Angle: ", angle)
    # Convert the angle to radians
    angle = np.radians(angle)

    for i in range(image.shape[1]):
        # Define the line parameters (angle and starting point)
        x0, y0 = i, 0  # Replace with your desired starting point

        # Calculate the end point of the line
        x1 = int(x0 + 200 * np.cos(angle))
        y1 = int(y0 + 200 * np.sin(angle))

        pixels = get_line_pixels_vertical(image, x0, y0, x1, y1)
        pixels = np.array(pixels, dtype=np.int16)

        if len(pixels) == 0:
            continue

        total_contrast = pixels[:, 0] - pixels[:, 1]
        all_contrasts.append(total_contrast.mean())

    all_contrasts = np.array(all_contrasts)
    # print(all_contrasts.argmin(), all_contrasts.min())
    # print(all_contrasts.argmax(), all_contrasts.max())
    length = all_contrasts.shape[0]
    argmin = all_contrasts[:40].argmin()
    min = all_contrasts[:40].min()
    argmax = all_contrasts[length-40:].argmax() + length-40
    max = all_contrasts[length-40:].max()
    all_values.append((argmin, min, argmax, max, max-min))

In [346]:
all_values = np.array(all_values)
best_lines = all_values[all_values[:, 4].argmax()]

# Define the line parameters (angle and starting point)
angle = 80 + all_values[:, 4].argmax()
x0_left, y0_left = int(best_lines[0]), 0
x0_right, y0_right = int(best_lines[2]), 0

print("Angle: ", angle)
print(best_lines)

# Convert the angle to radians
angle = np.radians(angle)

# Calculate the end point of the lines
x1_left = int(x0_left + 200 * np.cos(angle))
y1_left = int(y0_left + 200 * np.sin(angle))
x1_right = int(x0_right + 200 * np.cos(angle))
y1_right = int(y0_right + 200 * np.sin(angle))

image2 = image.copy()

cv2.line(image2, (x0_left, y0_left), (x1_left, y1_left), 255, 1)
cv2.line(image2, (x0_left+1, y0_left), (x1_left+1, y1_left), 0, 1)
cv2.line(image2, (x0_left-1, y0_left), (x1_left-1, y1_left), 0, 1)

cv2.line(image2, (x0_right, y0_right), (x1_right, y1_right), 255, 1)
cv2.line(image2, (x0_right+1, y0_right), (x1_right+1, y1_right), 0, 1)
cv2.line(image2, (x0_right-1, y0_right), (x1_right-1, y1_right), 0, 1)

cv2.imshow('image', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Angle:  83
[ 30.         -38.99107143 252.          48.24107143  87.23214286]
